In [1]:
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

In [16]:
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
df = pd.read_csv('heart.csv')

In [3]:
X = df.drop('target', axis=1)
y = df['target']

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

In [14]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)

RandomForestClassifier(random_state=42)

In [17]:
print(classification_report(y_test, model.predict(scaler.transform(X_test))))

              precision    recall  f1-score   support

           0       0.83      0.83      0.83        29
           1       0.84      0.84      0.84        32

    accuracy                           0.84        61
   macro avg       0.84      0.84      0.84        61
weighted avg       0.84      0.84      0.84        61



In [18]:
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

In [6]:
!pip install psycopg2-binary

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 19.0 MB/s  0:00:00 eta 0:00:01


In [7]:
import psycopg2

# RDS Bilgileriniz
DB_CONFIG = {
    "host": "database-1.c814i00i8t9k.us-east-1.rds.amazonaws.com",
    "database": "postgres",
    "user": "postgres", # veya oluşturduğunuz kullanıcı
    "password": "Bekobeko42"
}

try:
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()
    
    # Blackboard Tablosu: Tüm uzmanlar (KS) buraya bakar
    cur.execute("""
    CREATE TABLE IF NOT EXISTS heart_blackboard (
        id SERIAL PRIMARY KEY,
        patient_data JSONB,           -- Ham veriler (Eren Kaan'ın datasetinden gelen satır)
        ml_prediction JSONB,          -- Knowledge Source 1'in (ML) sonucu
        clinical_rules JSONB,         -- Knowledge Source 2'nin (Kural) sonucu
        status VARCHAR(20) DEFAULT 'PENDING', -- PENDING, PROCESSING, COMPLETED
        final_diagnosis TEXT,
        updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    );
    """)
    conn.commit()
    print("Blackboard (RDS) başarıyla hazırlandı!")
except Exception as e:
    print(f"Bağlantı Hatası: {e}")

Blackboard (RDS) başarıyla hazırlandı!


In [10]:
conn.rollback()
print("Transaction sıfırlandı, şimdi tabloyu yeniden oluşturabilirsiniz.")

Transaction sıfırlandı, şimdi tabloyu yeniden oluşturabilirsiniz.


In [11]:
# Mevcut tabloyu temizleyip yeniden kurma
cur.execute("DROP TABLE IF EXISTS heart_blackboard;")

cur.execute("""
CREATE TABLE heart_blackboard (
    id SERIAL PRIMARY KEY,
    input_data JSONB,           -- Ham veriler buraya gelecek
    ml_analysis JSONB,          -- Modül 1 (ML) sonucu
    clinical_analysis JSONB,    -- Modül 2 (Kural) sonucu
    status VARCHAR(20) DEFAULT 'PENDING',
    final_diagnosis TEXT,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
""")
conn.commit()
print("Tablo 'input_data' sütunu ile yeniden oluşturuldu.")

Tablo 'input_data' sütunu ile yeniden oluşturuldu.


In [12]:
for index, row in df.head(5).iterrows():
    patient_json = row.to_json()
    cur.execute(
        "INSERT INTO heart_blackboard (input_data, status) VALUES (%s, %s)",
        (patient_json, 'PENDING')
    )
conn.commit()
print("Veriler Blackboard'a (RDS) başarıyla yüklendi!")

Veriler Blackboard'a (RDS) başarıyla yüklendi!


In [19]:
import json

def run_diagnosis_system(patient_id):
    # 1. Blackboard'dan veriyi al
    cur.execute("SELECT input_data FROM heart_blackboard WHERE id = %s", (patient_id,))
    row = cur.fetchone()
    if not row: return
    
    patient_raw = row[0] # JSON formatındaki ham veri
    
    # 2. Knowledge Source 1 (ML Expert) çalıştır
    # Eren Kaan'ın eğittiği modeli kullanarak tahmin yap
    input_df = pd.DataFrame([patient_raw])
    scaled_input = scaler.transform(input_df)
    prediction = model.predict(scaled_input)[0]
    prob = model.predict_proba(scaled_input)[0][1]
    
    # Sonucu tahtaya (Blackboard) yaz
    cur.execute("UPDATE heart_blackboard SET ml_analysis = %s WHERE id = %s", 
                (json.dumps({"target": int(prediction), "probability": float(prob)}), patient_id))
    
    # 3. Knowledge Source 2 (Clinical Expert) çalıştır
    # Berke'nin kural tabanlı mantığı
    risk_level = "High" if patient_raw['chol'] > 240 or patient_raw['trestbps'] > 140 else "Normal"
    
    # Sonucu tahtaya yaz
    cur.execute("UPDATE heart_blackboard SET clinical_analysis = %s, status = 'COMPLETED' WHERE id = %s", 
                (json.dumps({"risk_score": risk_level}), patient_id))
    
    conn.commit()
    print(f"Teşhis tamamlandı: Hasta ID {patient_id}")

In [20]:
cur.execute("SELECT * FROM heart_blackboard;")
rows = cur.fetchall()
for row in rows:
    print(row)

(1, {'ca': 0.0, 'cp': 3.0, 'age': 63.0, 'fbs': 1.0, 'sex': 1.0, 'chol': 233.0, 'thal': 1.0, 'exang': 0.0, 'slope': 0.0, 'target': 1.0, 'oldpeak': 2.3, 'restecg': 0.0, 'thalach': 150.0, 'trestbps': 145.0}, None, None, 'PENDING', None, datetime.datetime(2025, 12, 19, 23, 45, 3, 84252))
(2, {'ca': 0.0, 'cp': 2.0, 'age': 37.0, 'fbs': 0.0, 'sex': 1.0, 'chol': 250.0, 'thal': 2.0, 'exang': 0.0, 'slope': 0.0, 'target': 1.0, 'oldpeak': 3.5, 'restecg': 1.0, 'thalach': 187.0, 'trestbps': 130.0}, None, None, 'PENDING', None, datetime.datetime(2025, 12, 19, 23, 45, 3, 84252))
(3, {'ca': 0.0, 'cp': 1.0, 'age': 41.0, 'fbs': 0.0, 'sex': 0.0, 'chol': 204.0, 'thal': 2.0, 'exang': 0.0, 'slope': 2.0, 'target': 1.0, 'oldpeak': 1.4, 'restecg': 0.0, 'thalach': 172.0, 'trestbps': 130.0}, None, None, 'PENDING', None, datetime.datetime(2025, 12, 19, 23, 45, 3, 84252))
(4, {'ca': 0.0, 'cp': 1.0, 'age': 56.0, 'fbs': 0.0, 'sex': 1.0, 'chol': 236.0, 'thal': 2.0, 'exang': 0.0, 'slope': 2.0, 'target': 1.0, 'oldpeak':